In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
import cv2
from matplotlib.image import imread
import os
import glob
import matplotlib.image as mpimg
from pathlib import Path
import seaborn as sns
from tqdm import tqdm
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks  import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from PIL import Image
import numpy as np
import cv2
import matplotlib.pylab as plt
from matplotlib.pyplot import figure
import tensorflow as tf
import random
from  sklearn.utils import shuffle
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from keras.layers import Input, Add, Dense,GlobalAvgPool2D
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.models import Model
from keras.layers import Input, Add, Dense, Concatenate, AvgPool2D, Dropout,BatchNormalization,  GlobalAveragePooling2D

Food images

In [ ]:
plt.figure(figsize=(10,10))
food_images="../input/food5k-image-dataset/training/food/"
for i in range(12):
    file=random.choice(os.listdir(food_images))
    food_image_path=os.path.join(food_images,file)
    img=mpimg.imread(food_image_path)
    ax=plt.subplot(3,4,i+1)
    plt.imshow(img)
plt.show()

Non Food images

In [ ]:
plt.figure(figsize=(10,10))
non_food_images="../input/food5k-image-dataset/training/non_food/"
for i in range(12):
    file=random.choice(os.listdir(non_food_images))
    non_food_image_path=os.path.join(non_food_images,file)
    img=mpimg.imread(non_food_image_path)
    ax=plt.subplot(3,4,i+1)
    plt.imshow(img)
plt.show()

In [ ]:
image_path=Path("../input/food5k-image-dataset/training/")
img_path="../input/food5k-image-dataset/training/"

In [ ]:
label_name=["Food","Non_Food"]

In [ ]:
image_size=(224,224)

In [ ]:
class_names = os.listdir(image_path)
print(class_names)
print("Number of classes : {}".format(len(class_names)))

In [ ]:
numberof_images={}
for class_name in class_names:
    numberof_images[class_name]=len(os.listdir(img_path+"/"+class_name))
images_each_class=pd.DataFrame(numberof_images.values(),index=numberof_images.keys(),columns=["Number of images"])
images_each_class

In [ ]:
# for folder_path, _, filenames in os.walk("../input/food5k-image-dataset/training/"):
#     for filename in filenames:
#         print(os.path.join(folder_path,filename))

Train Data

In [ ]:
train_data_path=Path(r"../input/food5k-image-dataset/training/")
image_data_path=list(train_data_path.glob(r"**/*.jpg"))
train_label_path=list(map(lambda x:os.path.split(os.path.split(x)[0])[1],image_data_path))
final_train_data=pd.DataFrame({"image_data":image_data_path,"label":train_label_path}).astype("str")
final_train_data=final_train_data.sample(frac=1).reset_index(drop=True)


In [ ]:
final_train_data

Valid Data

In [ ]:
valid_data_path=Path(r"../input/food5k-image-dataset/validation/")
image_data_path=list(valid_data_path.glob(r"**/*.jpg"))
valid_label_path=list(map(lambda x:os.path.split(os.path.split(x)[0])[1],image_data_path))
final_valid_data=pd.DataFrame({"image_data":image_data_path,"label":valid_label_path}).astype("str")
final_valid_data=final_valid_data.sample(frac=1).reset_index(drop=True)

In [ ]:
final_valid_data

Test Data

In [ ]:
test_data_path=Path(r"../input/food5k-image-dataset/evaluation/")
image_data_path=list(test_data_path.glob(r"**/*.jpg"))
test_label_path=list(map(lambda x:os.path.split(os.path.split(x)[0])[1],image_data_path))
final_test_data=pd.DataFrame({"image_data":image_data_path,"label":test_label_path}).astype("str")
final_test_data=final_test_data.sample(frac=1).reset_index(drop=True)

In [ ]:
final_test_data

In [ ]:
batch_size=30

In [ ]:
traindata_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,
                                                                    shear_range=0.2, horizontal_flip=True,validation_split=0.2,fill_mode='nearest')

validdata_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
testdata_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_generator=traindata_generator.flow_from_dataframe(dataframe=final_train_data,
                                                             x_col="image_data",
                                                             y_col="label",
                                                             batch_size=batch_size,
                                                             class_mode="categorical",
                                                             target_size=(224,224),
                                                             color_mode="rgb",
                                                             shuffle=True )

In [ ]:
valid_data_generator=validdata_generator.flow_from_dataframe(dataframe=final_valid_data,
                                                             x_col="image_data",
                                                             y_col="label",
                                                             batch_size=batch_size,class_mode="categorical",
                                                             target_size=(224,224),color_mode="rgb",shuffle=True )

In [ ]:
test_data_generator=testdata_generator.flow_from_dataframe(dataframe=final_test_data,
                                                           x_col="image_data",
                                                           y_col="label",
                                                           batch_size=batch_size,class_mode="categorical",
                                                           target_size=(224,224),color_mode="rgb",shuffle=False )

In [ ]:
class_dict = train_data_generator.class_indices
class_list = list(class_dict.keys())
class_list

In [ ]:
train_number=train_data_generator.samples
valid_number=valid_data_generator.samples

In [ ]:

dense121_model= tf.keras.applications.densenet.DenseNet121(weights='imagenet',include_top=False, input_shape=(224,224, 3))
x= dense121_model.output
x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(1024,activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x) 
x= Dense(512,activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x) 

x= Dropout(0.5)(x)
prediction= Dense(2, activation = 'softmax')(x)
model= Model(inputs= dense121_model.input, outputs= prediction)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
tensor_board=TensorBoard(log_dir="logs")
check_point=ModelCheckpoint("denseNet121.h5",monitor="val_accuracy",mode="auto",verbose=1,save_best_only=True)

In [ ]:
reduce_lr=ReduceLROnPlateau(monitor="val_accuracy",factor=0.3,patience=50,min_delta=0.001,mode="auto",verbose=1)

In [ ]:
history= model.fit(train_data_generator, 
                   steps_per_epoch=train_number//batch_size, 
                   validation_data= valid_data_generator, 
                   validation_steps= valid_number//batch_size,
                   shuffle=True, 
                   
                   epochs =40, 
                   batch_size = 30,callbacks=[tensor_board,check_point,reduce_lr])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_data_generator.class_indices


In [ ]:
prediction=model.predict(test_data_generator)
prediction=np.argmax(prediction,axis=1)
map_label=dict((m,n) for n,m in (test_data_generator.class_indices).items())
final_predict=pd.Series(prediction).map(map_label).values
y_test=list(final_test_data.label)

Confusion Matrix

In [ ]:

cm = confusion_matrix(y_test,final_predict)
sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu",xticklabels=label_name,yticklabels=label_name)
plt.title("Confusion Matrix\n")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()


Classification Report

In [ ]:
print(classification_report(y_test,final_predict))

Data Prediction

In [ ]:
plt.figure(figsize=(15,15))
number_images=(5,5)
for i in range(1,(number_images[0]*number_images[1])+1):
    plt.subplot(number_images[0],number_images[1],i)
    plt.axis("off")
    
    color="green"
    if final_test_data.label.iloc[i] !=final_predict[i]:
        color="red"
    plt.title(f"True:{final_test_data.label.iloc[i]}\nPredicted:{final_predict[i]}",color=color)
    plt.imshow(plt.imread(final_test_data['image_data'].iloc[i]))

Custom data prediction

In [ ]:
results={0:'Food',    1:'Non Food'}


test_image_path=Image.open("../input/food-test-images/baked-chicken-wings-asian-style-tomatoes-sauce-plate_2829-10657.jpg")
imge=test_image_path.resize(image_size)
imge=np.expand_dims(imge,axis=0)
imge=np.array(imge)/255
final_predct=np.argmax(model.predict([imge])[0])


        
print(results[final_predct])
plt.imshow(test_image_path)
plt.axis("off")
plt.show()